# 下载大模型

> 除非特殊说明，模型基本都采用Qwen模型进行测试，会使用base、instruct或者reasoning模型进行对比实验。

运行下面的代码可以将开源模型下载到本地，***如果使用API方式，不需要下载到本地***

为快速、顺利安装，我们使用modelscope，也就是魔搭社区中的模型下载到本地



In [ ]:
!modelscope download --model Qwen/Qwen2.5-3B  --local_dir /your/path/of/Qwen2.5-3B  # 下载base模型
!modelscope download --model Qwen/Qwen2.5-3B-Instruct  --local_dir /your/path/of/Qwen2.5-3B-Instruct  # 下载instruct模型
!modelscope download --model Qwen/Qwen3-1.7B  --local_dir /your/path/of/Qwen3-1.7B  # 下载reasoning模型
!modelscope download --model Qwen/Qwen2.5-VL-3B-Instruct  --local_dir /your/path/of/Qwen2.5-VL-3B-Instruct  # 下载VL模型
!modelscope download --model Qwen/Qwen2.5-7B  --local_dir /home/lixinyu/weights/Qwen2.5-7B  # 下载7B模型

# Huggingface&Modelscope

huggingface或者modelscope上模型发布的时候一般会自带推理代码，比如Qwen模型，推理代码在这👉[Qwen2.5推理代码](https://huggingface.co/Qwen/Qwen2.5-3B)。

我们只需要根据自身需要修改模型名称或者地址就可以实现**本地**的简单推理。

我们使用的是huggingface的transformers库，所以需要先安装transformers库。



In [ ]:
!pip install transformers

In [ ]:
### 加载模型
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name_or_path = 'Qwen/Qwen2.5-3B'  # 可以替换为你下载的模型路径
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,device_map='auto', torch_dtype='auto')

### 提示词
prompt = "Hello, Who are you?"

### 推理代码
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

# vllm

vLLM是一个快速且易于使用的库，用于 LLM 推理和服务，可以和HuggingFace 无缝集成。vLLM利用了全新的注意力算法「PagedAttention」，有效地管理注意力键和值。

`vllm在吞吐量方面，vLLM的性能比HuggingFace Transformers(HF)高出24倍，文本生成推理（TGI）高出3.5倍。`

简单点说就是vllm框架的推理速度很快，但是显存占用较高，同样的3B模型，本地推理可能只需要15GB左右，而vllm框架则需要37GB，因此如果硬件资源不足，vllm并不是一个很好的选择。

首先，我们需要在环境里安装vllm库



In [ ]:
!pip install vllm

然后我们在使用的时候主要参考的是推理时兼容OpenAI接口的情况，详细可以参考👉[中文文档](https://vllm.hyper.ai/docs/inference-and-serving/openai_compatible_server/)。

按照下面的步骤依次运行代码

1. 在本地默认端口跑本地模型，开启一个终端，运行下面的代码
    ```bash
    vllm serve /your/path/of/model
    ```
2. 开启新的终端页面运行各个代码。在新的终端页面，我们就可以跑我们对应的服务了，需要注意的是，我们使用大模型来进行推理的时候可以使用[openai的prompt的API的接口](https://openai.apifox.cn/api-55352401)或者[messages对应的API接口](https://openai.apifox.cn/api-67883981)等，输入接口参考给出的文档即可

    ```bash
    python your_code.py
    ```

# API

大模型的 API 调用，简单来说，是指开发者或用户通过应用程序编程接口（API） 与大模型进行交互，从而利用大模型的能力完成特定任务的进程，这些大模型无需部署到本地，你使用的资源其实是这些厂商提供的服务集群，不仅推理速度快，而且并不占用显存，不过每个模型会有费用的消耗，不一定能免费使用。

我们可以使用很多资源，这里我们简单举两个例子，分别是硅基流动和阿里百炼，可以参考他们的官网文档等：
1. 硅基流动：[官网](https://cloud.siliconflow.cn/sft-d1n0sv33jrms738gmgpg/models)
2. 阿里百炼：[API](https://bailian.console.aliyun.com/?spm=5176.12818093_47.resourceCenter.1.223c2cc96V9eQn&tab=api#/api/?type=model&url=https%3A%2F%2Fhelp.aliyun.com%2Fdocument_detail%2F2712576.html)

In [ ]:
import requests

url = "https://api.siliconflow.cn/v1/chat/completions"
api_key="sk-"

payload = {
    "model": "Qwen/Qwen3-8B",
    "messages": [
        {
            "role": "user",
            "content": "你好"
        }
    ],
    "enable_thinking":False
}
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"id":"0197f230c1437d020e874c6bff06fec3","object":"chat.completion","created":1752114971,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"你好！很高兴见到你。有什么我可以帮助你的吗？"},"finish_reason":"stop"}],"usage":{"prompt_tokens":13,"completion_tokens":12,"total_tokens":25,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""}


In [11]:
import json
result = json.loads(response.text)
print(result['choices'][0]['message']['content'])

你好！很高兴见到你。有什么我可以帮助你的吗？
